## 前期准备preparation in advance

In [1]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, TensorDataset
import random
import time
import pandas
import matplotlib.pyplot as plt

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
# Set random seeds to ensure reproducibility
# 设置随机种子以保证可复现
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)

set_seed(42)

In [3]:
#加载数据
def load_data(folder_path):
    # Retrieve all CSV files from the folder
    # 获取文件夹中的所有 CSV 文件
    csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]  
    # List used to store data
    # 用来存放数据的列表
    data_list = []  
    
    for file_name in csv_files:
        file_path = os.path.join(folder_path, file_name)
        # Read CSV file
        # 读取 CSV 文件
        df = pd.read_csv(file_path,encoding='gbk') 
        #Only the first and tenth columns of data (including label data) have been selected here and can be freely adjusted
        #这里只选中了第1列与第10列数据（包括标签数据），可自由调整
        select_data=df.iloc[:,1:10]
        #Assuming that the number of rows in each file is seq_1en and the number of columns is feature_stize
        # 假设每个文件的行数是 seq_len，列数是 feature_size
        seq_len = select_data.shape[0]  
        feature_size = select_data.shape[1]  
        #Convert DataFrame to NumPy array
        # 将 DataFrame 转换为 NumPy 数组
        data = select_data.to_numpy()
        #Add data to data_ist
        # 将数据添加到 data_list 中
        data_list.append(data)
    #Concatenate the data of all files into a large NumPy array
    # 将所有文件的数据拼接成一个大的 NumPy 数组
    data_array = np.array(data_list)  # shape: (site_size, seq_len, feature_size)
    data_array = data_array.transpose(1,0,2)# shape: (seq_len, site_size, feature_size)
    return data_array
folder_path = 'E:/your/data/folder_path'
data = load_data(folder_path)
print(data.shape)
#data

(30642, 9, 9)


In [4]:
# EarlyStopping，patience:allowing the maximum number of training epochs without improvement in validation loss; delta： When the validation loss improves delta at least compared to the optimal loss, it is considered to have improved
# 早停机制,patience验证损失未改善情况下允许训练的最大次数；delta：当验证损失比最佳损失至少改善delta则认为有所改善
class EarlyStopping:
    def __init__(self, patience=10, delta=0):
        self.patience = patience
        self.delta = delta
        self.best_loss = float('inf')
        self.counter = 0
        self.early_stop = False

    def __call__(self, val_loss):
        if val_loss < self.best_loss - self.delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

In [5]:
#Create a sliding window
#制作滑动窗口
def create_sequences(data, input_seq_len, output_seq_len, target_site_index, target_col):
    X, y = [], []
    #Traverse all possible sequences
    # 遍历所有可能的序列
    for i in range(len(data) - input_seq_len - output_seq_len + 1):  # len(data) 是 seq_len
        #Create input data，shape:(site_size=9, input_seq_len=72, feature_size=9)
        # 制作输入序列数据，形状是 (site_size=9, input_seq_len=72, feature_size=9)
        X_sample = data[i:i + input_seq_len,:,  :]  # 选择 seq_len 范围的数据
        X.append(X_sample)
        
        #Create output data，shape:(target_site_size=1, output_seq_len, target_feature_size=1)
        # 制作输出序列数据，形状是 (target_site_size=1, output_seq_len, target_feature_size=1)
        target_data = data[i + input_seq_len:i + input_seq_len + output_seq_len,target_site_index,  target_col]
        y.append(target_data.reshape(output_seq_len,1,  1))  
        
    return np.array(X), np.array(y)


X shape: (30570, 72, 9, 9)
y shape: (30570, 1)
X_batch shape: torch.Size([16, 72, 9, 9])
y_batch shape: torch.Size([16, 1])
torch.Size([16, 72, 9, 9])
torch.Size([16, 1])


In [7]:
# 定义LSTM模型
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)  # 批量计算
        #print(lstm_out.shape)
        out = self.fc(lstm_out[:, -1, :])  # 仅使用最后时间步的隐藏状态#(batch_size,seq_len,hidden_size)
        #out = self.fc(lstm_out[:, :output_seq_len, :])
        #print(out.shape)
        return out

GCN_LSTM_Model(
  (gcn): GCNConv(9, 1)
  (lstm): LSTM(18, 64, num_layers=2, batch_first=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (fc): Linear(in_features=64, out_features=1, bias=True)
)

In [8]:
#评估指标
from sklearn.metrics import mean_absolute_error, r2_score
import numpy as np

# 计算 RMSE、R2 和 MAE
def calculate_metrics(predictions, actuals):
    # 计算 RMSE
    rmse = np.sqrt(np.mean((predictions - actuals) ** 2))
    
    # 计算 R2
    r2 = r2_score(actuals, predictions)
    
    # 计算 MAE
    mae = mean_absolute_error(actuals, predictions)
    
    return rmse, r2, mae

## 定义函数（不同循环需修改）Define function (different loops need to be modified)

In [10]:
#Single site single feature input is X_batch = X_origin[:,:,target_site_index,input_feature].unsqueeze(-1),LSTM_input_size = 1
#单站点单特征输入为X_batch = X_origin[:,:,target_site_index,input_feature].unsqueeze(-1),LSTM_input_size = 1
#Single site all feature input is X_batch = X_origin[:,:,target_site_index,:].squeeze(),LSTM_input_size = 9
#单站点所有特征输入为X_batch = X_origin[:,:,target_site_index,:].squeeze(),LSTM_input_size = 9
#all site single feature input is X_batch = X_origin[:,:,:,input_feature].squeeze(),LSTM_input_size = 9
#所有站点单特征输入为X_batch = X_origin[:,:,:,input_feature].squeeze(),LSTM_input_size = 9
#all site all feature input is X_batch = X_origin.reshape(X_origin.size(0), X_origin.size(1), -1),LSTM_input_size = 81
#所有站点所有特征输入为X_batch = X_origin.reshape(X_origin.size(0), X_origin.size(1), -1),LSTM_input_size = 81
#Remove DO:X_batch = np.delete(X_batch,target_site_index,axis = 2)
#剔除DO：X_batch = np.delete(X_batch,target_site_index,axis = 2)

# input_batch = slice(None)
# input_seq = slice(None)
# input_site = target_site_index

# LSTM训练和验证循环
def LSTM_train_and_evaluate(lstm_model, train_loader, test_loader, criterion, optimizer, epochs, target_site_index, input_feature):
    early_stopping = EarlyStopping(patience=10, delta=0.001)
    best_model
    for epoch in range(epochs):
        start_time=time.time()
        lstm_model.train()
        train_loss = 0
        for X_origin, y_batch in train_loader:
            X_batch = X_origin[:,:,target_site_index,input_feature].squeeze()#(batch_size,seq_len, site_size, combined_feature_size)
#             X_batch = np.delete(X_batch,2,axis = 2)
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            #print(y_batch.shape)
            optimizer.zero_grad()
            outputs = lstm_model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        train_loss /= len(train_loader)

        # 验证
        lstm_model.eval()
        val_loss = 0
        with torch.no_grad():
            for X_origin, y_batch in val_loader:
                X_batch = X_origin[:,:,target_site_index,input_feature].squeeze()
#                 X_batch = np.delete(X_batch,2,axis = 2)
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                outputs = lstm_model(X_batch)
                loss = criterion(outputs, y_batch)
                val_loss += loss.item()

        val_loss /= len(val_loader)
        end_time=time.time()
        epoch_duration=end_time-start_time
        epoch_logs.append([epoch+1,train_loss,val_loss,epoch_duration])
        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

        early_stopping(val_loss)
        if early_stopping.early_stop:
            print("Early stopping triggered.")
            break

#train_and_evaluate(model, train_loader, val_loader, criterion, optimizer, epochs)

# LSTM测试集评估
def LSTM_evaluate(lstm_model, test_loader, target_site_index, input_feature):
    lstm_model.eval()
    predictions, actuals = [], []
    with torch.no_grad():
        for X_origin, y_batch in test_loader:
            X_batch = X_origin[:,:,target_site_index,input_feature].squeeze()
#             X_batch = np.delete(X_batch,2,axis = 2)
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = lstm_model(X_batch)
            predictions.append(outputs.cpu().numpy())  # 将数据移回CPU
            actuals.append(y_batch.cpu().numpy())      # 将数据移回CPU

    predictions = np.concatenate(predictions, axis=0)
    actuals = np.concatenate(actuals, axis=0)
    mse = mean_squared_error(actuals, predictions)
    print(f"Test MSE: {mse:.4f}")

    return predictions, actuals

#predictions, actuals = evaluate(model, test_loader)
#actuals.shape

#lstm model sensitivity
#LSTM特征重要性分析
def LSTM_feature_importance_analysis(lstm_model, test_loader, target_site_index, input_feature, baseline='zero'):
    """
    计算特征的重要性：对于每个特征，在其置零或加噪声时，观察对模型输出的影响。
    """
    feature_importances_all = []
    for X_origin, y_batch in test_loader:
        feature_importances = []
        X_batch = X_origin[:,:,target_site_index,input_feature].squeeze()
#         X_batch = np.delete(X_batch,2,axis = 2)
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        lstm_model.eval()
        baseline_output = lstm_model(X_batch).detach().cpu().numpy()  # 将数据移回CPU
        for j in range(X_batch.shape[2]):  # 遍历特征
            perturbed_output = []
            X_perturbed = X_batch.clone()
            if baseline == 'zero':
                X_perturbed[:, :, j] = 0  # 将第 i 个特征置零
            outputs = lstm_model(X_perturbed)
            perturbed_output.append(outputs.detach().cpu().numpy())
            importance = np.mean(np.abs(perturbed_output - baseline_output))
            feature_importances.append(importance)
        feature_importances_all.append(feature_importances)
    return feature_importances_all

## 单站点单特征Single site single feature

In [17]:
#单站点单特征循环Single site single feature
#change the Define function.input: X_origin[:,:,target_site_index,input_feature].unsqueeze(-1),LSTM_input_size = 1
all_output_metrics = []
all_output_importance = []

for model_sites in range(9):
    output_metrics = []
    output_importance = []
    for model_input_feature in range(9):
        #Model hyperparameters
        # 模型超参数
        LSTM_input_size = 1
        LSTM_hidden_size = 64  
        LSTM_num_layers = 2

        input_seq_len = 72  # 输入序列长度,Input sequence length
        output_seq_len = 1  # 输出序列长度,output sequence length
        target_site_index = model_sites  # 目标站点（例如 站点0）,Target site (e.g. site 0)
        target_col = 2  # 目标特征（例如 特征0）,Target feature (e.g. feature 0)
        batch_size = 16

        X, y = create_sequences(data, input_seq_len, output_seq_len, target_site_index, target_col)
        y = y.squeeze((-1,-2))
        #Divide the training set，validation set and test set 7:1:2
        # 划分训练集验证集和测试集7：1：2
        train_size, val_size, test_size = 0.7, 0.1, 0.2
        X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=test_size/(train_size+val_size+test_size), shuffle=False)
        X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=val_size/(train_size+val_size), shuffle=False)
        X_train, X_val, X_test = to_tensor(X_train), to_tensor(X_val), to_tensor(X_test)
        y_train, y_val, y_test = to_tensor(y_train), to_tensor(y_val), to_tensor(y_test)
        # create DataLoader
        train_dataset = TensorDataset(X_train, y_train)
        val_dataset = TensorDataset(X_val, y_val)
        test_dataset = TensorDataset(X_test, y_test)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

        lstm_model = LSTMModel(LSTM_input_size, LSTM_hidden_size, LSTM_num_layers, output_seq_len)
        lstm_model.to(device)

        epochs=200
        learning_rate = 0.001
        criterion = nn.MSELoss()
        optimizer = optim.Adam(lstm_model.parameters(), lr=learning_rate)
        epoch_logs=[]
        input_feature = model_input_feature#0:WT,1:pH,2:DO,3:EC,4:TSS,5:CODMN,6:AN,7:TP,8:TN

        LSTM_train_and_evaluate(lstm_model, train_loader, val_loader, criterion, optimizer, epochs, target_site_index, input_feature)
        predictions, actuals = LSTM_evaluate(lstm_model, test_loader, target_site_index, input_feature)
        #Calculate accuracy
        # 计算精度
        rmse, r2, mae = calculate_metrics(predictions, actuals)
        #result
        # 输出结果
        print(f"predictions shape: {predictions.shape}")
        print(f"actuals shape: {actuals.shape}")
        print(f"Test RMSE: {rmse:.4f}")
        print(f"Test R2: {r2:.4f}")
        print(f"Test MAE: {mae:.4f}")
        feature_importances = np.mean(pd.DataFrame(LSTM_feature_importance_analysis(lstm_model, test_loader, target_site_index, input_feature, baseline='zero')),axis= 0)
        print(feature_importances)
        
        output_metrics.append([rmse, r2, mae])
        output_importance.append(feature_importances)
        epoch_logs = pd.DataFrame(epoch_logs,columns=['Epoch','Train_Loss','Val_loss','Time'])
        epoch_logs.to_csv(f"LSTM_logs_1site_1feature_batch_site{model_sites}_feature{model_input_feature}.csv",index=False)
        
    all_output_metrics.append(pd.DataFrame(output_metrics,columns=['RMSE','R2','MAE']))
    all_output_importance.append(pd.DataFrame({f"N{model_sites}" : output_importance.values}, index = ['WT','pH','DO','EC','TSS','CODMn','AN','TP','TN']))
    
result_output_metrics = pd.concat(all_output_metrics,axis = 1)
result_output_importance = pd.concat(all_output_importance,axis = 1)
result_output_metrics.index = result_output_importance.index = ['WT','pH','DO','EC','TSS','CODMn','AN','TP','TN']

result_output_metrics.to_csv(f"LSTM_1site_1feature_batch_metrics.csv",index=False)
result_output_importance.to_csv(f"LSTM_1site_1feature_batch_importance.csv",index=False)
#['WT','pH','DO','EC','TSS','CODMn','AN','TP','TN']

<>:70: SyntaxWarning: list indices must be integers or slices, not tuple; perhaps you missed a comma?
<>:70: SyntaxWarning: list indices must be integers or slices, not tuple; perhaps you missed a comma?


Epoch 1/200, Train Loss: 3.2389, Val Loss: 0.9915
Epoch 2/200, Train Loss: 1.5500, Val Loss: 1.0420
Epoch 3/200, Train Loss: 1.5020, Val Loss: 1.0785
Epoch 4/200, Train Loss: 1.4448, Val Loss: 1.2363
Epoch 5/200, Train Loss: 1.3729, Val Loss: 1.0240
Epoch 6/200, Train Loss: 1.3396, Val Loss: 0.8871
Epoch 7/200, Train Loss: 1.3293, Val Loss: 0.8433
Epoch 8/200, Train Loss: 1.3114, Val Loss: 0.8371
Epoch 9/200, Train Loss: 1.2899, Val Loss: 0.7977
Epoch 10/200, Train Loss: 1.2793, Val Loss: 0.9560
Epoch 11/200, Train Loss: 1.2624, Val Loss: 0.8622
Epoch 12/200, Train Loss: 1.2326, Val Loss: 0.6730
Epoch 13/200, Train Loss: 1.2260, Val Loss: 0.6633
Epoch 14/200, Train Loss: 1.2076, Val Loss: 0.7069
Epoch 15/200, Train Loss: 1.1810, Val Loss: 0.7563
Epoch 16/200, Train Loss: 1.1673, Val Loss: 0.6417
Epoch 17/200, Train Loss: 1.1066, Val Loss: 0.7578
Epoch 18/200, Train Loss: 1.0565, Val Loss: 0.7873
Epoch 19/200, Train Loss: 1.0082, Val Loss: 0.6095
Epoch 20/200, Train Loss: 0.9628, Val Lo

Epoch 20/200, Train Loss: 2.9298, Val Loss: 1.6608
Epoch 21/200, Train Loss: 2.9441, Val Loss: 1.3515
Epoch 22/200, Train Loss: 2.9320, Val Loss: 2.0818
Epoch 23/200, Train Loss: 2.9373, Val Loss: 1.6360
Epoch 24/200, Train Loss: 2.9127, Val Loss: 1.8944
Epoch 25/200, Train Loss: 2.8910, Val Loss: 1.6593
Epoch 26/200, Train Loss: 2.8913, Val Loss: 1.9019
Epoch 27/200, Train Loss: 2.8788, Val Loss: 1.6140
Epoch 28/200, Train Loss: 2.8813, Val Loss: 1.7253
Epoch 29/200, Train Loss: 2.8378, Val Loss: 1.7940
Epoch 30/200, Train Loss: 2.8082, Val Loss: 1.4873
Epoch 31/200, Train Loss: 2.7832, Val Loss: 1.8840
Early stopping triggered.
Test MSE: 2.5992
predictions shape: (6114, 1)
actuals shape: (6114, 1)
Test RMSE: 1.6122
Test R2: -0.0806
Test MAE: 1.3563
0    0.330075
dtype: float32
Epoch 1/200, Train Loss: 4.4991, Val Loss: 2.1549
Epoch 2/200, Train Loss: 3.1224, Val Loss: 2.1408
Epoch 3/200, Train Loss: 3.1224, Val Loss: 1.8148
Epoch 4/200, Train Loss: 3.1184, Val Loss: 2.2299
Epoch 5/20

Epoch 5/200, Train Loss: 3.2978, Val Loss: 3.1882
Epoch 6/200, Train Loss: 3.3123, Val Loss: 4.2462
Epoch 7/200, Train Loss: 3.2805, Val Loss: 4.3137
Epoch 8/200, Train Loss: 3.2622, Val Loss: 2.9066
Epoch 9/200, Train Loss: 3.2404, Val Loss: 3.9027
Epoch 10/200, Train Loss: 3.2263, Val Loss: 3.2925
Epoch 11/200, Train Loss: 3.2094, Val Loss: 2.9100
Epoch 12/200, Train Loss: 3.2020, Val Loss: 4.7766
Early stopping triggered.
Test MSE: 1.7527
predictions shape: (6114, 1)
actuals shape: (6114, 1)
Test RMSE: 1.3239
Test R2: 0.0182
Test MAE: 1.1203
0    0.210704
dtype: float32
Epoch 1/200, Train Loss: 3.7991, Val Loss: 2.2493
Epoch 2/200, Train Loss: 2.6609, Val Loss: 2.1772
Epoch 3/200, Train Loss: 2.5249, Val Loss: 2.0116
Epoch 4/200, Train Loss: 2.4138, Val Loss: 2.0932
Epoch 5/200, Train Loss: 2.3195, Val Loss: 1.7767
Epoch 6/200, Train Loss: 2.2090, Val Loss: 1.6032
Epoch 7/200, Train Loss: 2.0238, Val Loss: 2.1891
Epoch 8/200, Train Loss: 1.8568, Val Loss: 1.5880
Epoch 9/200, Train L

Epoch 7/200, Train Loss: 0.1900, Val Loss: 0.1937
Epoch 8/200, Train Loss: 0.1885, Val Loss: 0.2100
Epoch 9/200, Train Loss: 0.1896, Val Loss: 0.2010
Epoch 10/200, Train Loss: 0.1863, Val Loss: 0.1830
Epoch 11/200, Train Loss: 0.1848, Val Loss: 0.1842
Epoch 12/200, Train Loss: 0.1841, Val Loss: 0.1949
Epoch 13/200, Train Loss: 0.1831, Val Loss: 0.2171
Epoch 14/200, Train Loss: 0.1824, Val Loss: 0.1914
Epoch 15/200, Train Loss: 0.1837, Val Loss: 0.2024
Epoch 16/200, Train Loss: 0.1796, Val Loss: 0.1818
Epoch 17/200, Train Loss: 0.1774, Val Loss: 0.2125
Epoch 18/200, Train Loss: 0.1747, Val Loss: 0.1693
Epoch 19/200, Train Loss: 0.1729, Val Loss: 0.1734
Epoch 20/200, Train Loss: 0.1705, Val Loss: 0.1739
Epoch 21/200, Train Loss: 0.1668, Val Loss: 0.1709
Epoch 22/200, Train Loss: 0.1667, Val Loss: 0.1580
Epoch 23/200, Train Loss: 0.1647, Val Loss: 0.1504
Epoch 24/200, Train Loss: 0.1618, Val Loss: 0.1436
Epoch 25/200, Train Loss: 0.1643, Val Loss: 0.1481
Epoch 26/200, Train Loss: 0.1605, 

...

Epoch 4/200, Train Loss: 1.5998, Val Loss: 1.1690
Epoch 5/200, Train Loss: 1.5871, Val Loss: 1.0787
Epoch 6/200, Train Loss: 1.5798, Val Loss: 1.1853
Epoch 7/200, Train Loss: 1.5591, Val Loss: 1.3331
Epoch 8/200, Train Loss: 1.5536, Val Loss: 1.1192
Epoch 9/200, Train Loss: 1.6061, Val Loss: 1.2035
Epoch 10/200, Train Loss: 1.5404, Val Loss: 1.1202
Epoch 11/200, Train Loss: 1.5007, Val Loss: 1.2354
Epoch 12/200, Train Loss: 1.4720, Val Loss: 1.4656
Epoch 13/200, Train Loss: 1.4546, Val Loss: 1.1811
Epoch 14/200, Train Loss: 1.4083, Val Loss: 1.1813
Epoch 15/200, Train Loss: 1.4144, Val Loss: 1.2845
Early stopping triggered.
Test MSE: 2.5173
predictions shape: (6114, 1)
actuals shape: (6114, 1)
Test RMSE: 1.5866
Test R2: -0.0249
Test MAE: 1.3024
0    2.122486
dtype: float32
Epoch 1/200, Train Loss: 2.0116, Val Loss: 1.1353
Epoch 2/200, Train Loss: 1.5155, Val Loss: 1.1430
Epoch 3/200, Train Loss: 1.5113, Val Loss: 1.1334
Epoch 4/200, Train Loss: 1.5045, Val Loss: 1.1430
Epoch 5/200, Tra

Epoch 30/200, Train Loss: 1.7747, Val Loss: 1.2035
Epoch 31/200, Train Loss: 1.7418, Val Loss: 1.2288
Epoch 32/200, Train Loss: 1.7036, Val Loss: 1.2871
Epoch 33/200, Train Loss: 1.7287, Val Loss: 1.2805
Early stopping triggered.
Test MSE: 1.9666
predictions shape: (6114, 1)
actuals shape: (6114, 1)
Test RMSE: 1.4023
Test R2: 0.1455
Test MAE: 1.0673
0    0.507165
dtype: float32
Epoch 1/200, Train Loss: 2.5573, Val Loss: 1.4299
Epoch 2/200, Train Loss: 2.1374, Val Loss: 1.3785
Epoch 3/200, Train Loss: 2.1395, Val Loss: 1.5282
Epoch 4/200, Train Loss: 2.1389, Val Loss: 1.5073
Epoch 5/200, Train Loss: 2.1395, Val Loss: 1.3643
Epoch 6/200, Train Loss: 2.1366, Val Loss: 1.4904
Epoch 7/200, Train Loss: 2.1393, Val Loss: 1.4397
Epoch 8/200, Train Loss: 2.1385, Val Loss: 1.3680
Epoch 9/200, Train Loss: 2.1367, Val Loss: 1.2844
Epoch 10/200, Train Loss: 2.1367, Val Loss: 1.4722
Epoch 11/200, Train Loss: 2.1393, Val Loss: 1.3299
Epoch 12/200, Train Loss: 2.1406, Val Loss: 1.4860
Epoch 13/200, Tr

Epoch 7/200, Train Loss: 2.6124, Val Loss: 5.5899
Epoch 8/200, Train Loss: 2.4478, Val Loss: 4.2727
Epoch 9/200, Train Loss: 2.2765, Val Loss: 4.6957
Epoch 10/200, Train Loss: 2.0230, Val Loss: 5.3010
Epoch 11/200, Train Loss: 1.8271, Val Loss: 6.7253
Epoch 12/200, Train Loss: 1.6408, Val Loss: 4.5143
Epoch 13/200, Train Loss: 1.4536, Val Loss: 3.5278
Epoch 14/200, Train Loss: 1.3632, Val Loss: 5.3291
Epoch 15/200, Train Loss: 1.2904, Val Loss: 4.9914
Epoch 16/200, Train Loss: 1.1643, Val Loss: 3.5733
Epoch 17/200, Train Loss: 1.1139, Val Loss: 6.9710
Epoch 18/200, Train Loss: 1.0622, Val Loss: 6.1861
Epoch 19/200, Train Loss: 0.9959, Val Loss: 6.5969
Epoch 20/200, Train Loss: 0.9177, Val Loss: 4.5631
Epoch 21/200, Train Loss: 0.8922, Val Loss: 6.1101
Epoch 22/200, Train Loss: 0.8698, Val Loss: 5.2171
Epoch 23/200, Train Loss: 0.8330, Val Loss: 4.8223
Early stopping triggered.
Test MSE: 3.7877
predictions shape: (6114, 1)
actuals shape: (6114, 1)
Test RMSE: 1.9462
Test R2: -0.0163
Test

C:\Users\18256\AppData\Local\Temp\ipykernel_21256\1127783596.py:70: SyntaxWarning: list indices must be integers or slices, not tuple; perhaps you missed a comma?
  result_output_metrics.index = result_output_importance.index = ['WT','pH','DO','EC','TSS','CODMn','AN','TP','TN']['WT','pH','DO','EC','TSS','CODMn','AN','TP','TN']
C:\Users\18256\AppData\Local\Temp\ipykernel_21256\1127783596.py:70: SyntaxWarning: list indices must be integers or slices, not tuple; perhaps you missed a comma?
  result_output_metrics.index = result_output_importance.index = ['WT','pH','DO','EC','TSS','CODMn','AN','TP','TN']['WT','pH','DO','EC','TSS','CODMn','AN','TP','TN']
C:\Users\18256\AppData\Local\Temp\ipykernel_21256\1127783596.py:70: SyntaxWarning: list indices must be integers or slices, not tuple; perhaps you missed a comma?
  result_output_metrics.index = result_output_importance.index = ['WT','pH','DO','EC','TSS','CODMn','AN','TP','TN']['WT','pH','DO','EC','TSS','CODMn','AN','TP','TN']


TypeError: list indices must be integers or slices, not tuple

In [33]:
#test
all_output_metrics = []
all_output_importance = []

for model_sites in range(9):
    output_metrics = []
    output_importance = []
    for model_input_feature in range(1):
        #Model hyperparameters
        # 模型超参数
        LSTM_input_size = 1
        LSTM_hidden_size = 64  
        LSTM_num_layers = 2

        input_seq_len = 72  # 输入序列长度,Input sequence length
        output_seq_len = 1  # 输出序列长度,output sequence length
        target_site_index = model_sites  # 目标站点（例如 站点0）,Target site (e.g. site 0)
        target_col = 2  # 目标特征（例如 特征0）,Target feature (e.g. feature 0)
        batch_size = 16

        X, y = create_sequences(data, input_seq_len, output_seq_len, target_site_index, target_col)
        y = y.squeeze((-1,-2))
        #Divide the training set，validation set and test set 7:1:2
        # 划分训练集验证集和测试集7：1：2
        train_size, val_size, test_size = 0.7, 0.1, 0.2
        X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=test_size/(train_size+val_size+test_size), shuffle=False)
        X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=val_size/(train_size+val_size), shuffle=False)
        X_train, X_val, X_test = to_tensor(X_train), to_tensor(X_val), to_tensor(X_test)
        y_train, y_val, y_test = to_tensor(y_train), to_tensor(y_val), to_tensor(y_test)
        # create DataLoader
        train_dataset = TensorDataset(X_train, y_train)
        val_dataset = TensorDataset(X_val, y_val)
        test_dataset = TensorDataset(X_test, y_test)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

        lstm_model = LSTMModel(LSTM_input_size, LSTM_hidden_size, LSTM_num_layers, output_seq_len)
        lstm_model.to(device)

        epochs=200
        learning_rate = 0.001
        criterion = nn.MSELoss()
        optimizer = optim.Adam(lstm_model.parameters(), lr=learning_rate)
        epoch_logs=[]
        input_feature = model_input_feature#0:WT,1:pH,2:DO,3:EC,4:TSS,5:CODMN,6:AN,7:TP,8:TN

        LSTM_train_and_evaluate(lstm_model, train_loader, val_loader, criterion, optimizer, epochs, target_site_index, input_feature)
        predictions, actuals = LSTM_evaluate(lstm_model, test_loader, target_site_index, input_feature)
        #Calculate accuracy
        # 计算精度
        rmse, r2, mae = calculate_metrics(predictions, actuals)
        #result
        # 输出结果
        print(f"predictions shape: {predictions.shape}")
        print(f"actuals shape: {actuals.shape}")
        print(f"Test RMSE: {rmse:.4f}")
        print(f"Test R2: {r2:.4f}")
        print(f"Test MAE: {mae:.4f}")
        feature_importances = np.mean(pd.DataFrame(LSTM_feature_importance_analysis(lstm_model, test_loader, target_site_index, input_feature, baseline='zero')),axis= 0)
        print(feature_importances)
        
        output_metrics.append([rmse, r2, mae])
        output_importance.append(feature_importances)
        epoch_logs = pd.DataFrame(epoch_logs,columns=['Epoch','Train_Loss','Val_loss','Time'])
        epoch_logs.to_csv(f"LSTM_logs_1site_1feature_batch_site{model_sites}_feature{model_input_feature}.csv",index=False)
        
    all_output_metrics.append(pd.DataFrame(output_metrics,columns=['RMSE','R2','MAE']))
    all_output_importance.append(pd.DataFrame({f"N{model_sites}" : output_importance.values}, index = ['WT','pH','DO','EC','TSS','CODMn','AN','TP','TN']))
    
result_output_metrics = pd.concat(all_output_metrics,axis = 1)
result_output_importance = pd.concat(all_output_importance,axis = 1)
result_output_metrics.index = result_output_importance.index = ['WT','pH','DO','EC','TSS','CODMn','AN','TP','TN']

result_output_metrics.to_csv(f"LSTM_1site_1feature_batch_metrics.csv",index=False)
result_output_importance.to_csv(f"LSTM_1site_1feature_batch_importance.csv",index=False)
#['WT','pH','DO','EC','TSS','CODMn','AN','TP','TN']

Epoch 1/200, Train Loss: 3.1482, Val Loss: 0.0874
Epoch 2/200, Train Loss: 0.1274, Val Loss: 0.0882
Epoch 3/200, Train Loss: 0.1106, Val Loss: 0.0720
Epoch 4/200, Train Loss: 0.1056, Val Loss: 0.0879
Epoch 5/200, Train Loss: 0.1043, Val Loss: 0.0580
Epoch 6/200, Train Loss: 0.1009, Val Loss: 0.0822
Epoch 7/200, Train Loss: 0.0991, Val Loss: 0.0624
Epoch 8/200, Train Loss: 0.0972, Val Loss: 0.0646
Epoch 9/200, Train Loss: 0.0975, Val Loss: 0.0559
Epoch 10/200, Train Loss: 0.0960, Val Loss: 0.0641
Epoch 11/200, Train Loss: 0.0963, Val Loss: 0.0633
Epoch 12/200, Train Loss: 0.0946, Val Loss: 0.0650
Epoch 13/200, Train Loss: 0.0934, Val Loss: 0.0663
Epoch 14/200, Train Loss: 0.0939, Val Loss: 0.0631
Epoch 15/200, Train Loss: 0.0923, Val Loss: 0.0673
Epoch 16/200, Train Loss: 0.0905, Val Loss: 0.0704
Epoch 17/200, Train Loss: 0.0901, Val Loss: 0.0647
Epoch 18/200, Train Loss: 0.0894, Val Loss: 0.0686
Epoch 19/200, Train Loss: 0.0885, Val Loss: 0.0579
Early stopping triggered.
Test MSE: 0.07

Epoch 6/200, Train Loss: 0.1499, Val Loss: 0.0482
Epoch 7/200, Train Loss: 0.1474, Val Loss: 0.0667
Epoch 8/200, Train Loss: 0.1450, Val Loss: 0.0448
Epoch 9/200, Train Loss: 0.1428, Val Loss: 0.0486
Epoch 10/200, Train Loss: 0.1411, Val Loss: 0.0492
Epoch 11/200, Train Loss: 0.1423, Val Loss: 0.0457
Epoch 12/200, Train Loss: 0.1415, Val Loss: 0.0451
Epoch 13/200, Train Loss: 0.1391, Val Loss: 0.0439
Epoch 14/200, Train Loss: 0.1396, Val Loss: 0.0477
Epoch 15/200, Train Loss: 0.1374, Val Loss: 0.0470
Epoch 16/200, Train Loss: 0.1367, Val Loss: 0.0477
Epoch 17/200, Train Loss: 0.1365, Val Loss: 0.0461
Epoch 18/200, Train Loss: 0.1366, Val Loss: 0.0468
Early stopping triggered.
Test MSE: 0.0502
predictions shape: (6114, 1)
actuals shape: (6114, 1)
Test RMSE: 0.2241
Test R2: 0.9844
Test MAE: 0.1416
0    5.33042
dtype: float32
Epoch 1/200, Train Loss: 1.2074, Val Loss: 0.1534
Epoch 2/200, Train Loss: 0.3074, Val Loss: 0.1453
Epoch 3/200, Train Loss: 0.3078, Val Loss: 0.1461
Epoch 4/200, Tr

## 单站点所有特征Single site all feature

In [39]:
#单站点所有特征循环,Single site all feature
#change the Define function input:X_origin[:,:,target_site_index,:].squeeze(),LSTM_input_size = 9
all_output_metrics = []
all_output_importance = []

for model_sites in range(9):
    #Model hyperparameters
    # 模型超参数
    LSTM_input_size = 9
    LSTM_hidden_size = 64  
    LSTM_num_layers = 2

    input_seq_len = 72  # 输入序列长度,Input sequence length
    output_seq_len = 1  # 输出序列长度,output sequence length
    target_site_index = model_sites  # 目标站点（例如 站点0）,Target site (e.g. site 0)
    target_col = 2  # 目标特征（例如 特征0）,Target feature (e.g. feature 0)
    batch_size = 16

    X, y = create_sequences(data, input_seq_len, output_seq_len, target_site_index, target_col)
    y = y.squeeze((-1,-2))
    #Divide the training set，validation set and test set 7:1:2
    # 划分训练集验证集和测试集7：1：2
    train_size, val_size, test_size = 0.7, 0.1, 0.2
    X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=test_size/(train_size+val_size+test_size), shuffle=False)
    X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=val_size/(train_size+val_size), shuffle=False)
    X_train, X_val, X_test = to_tensor(X_train), to_tensor(X_val), to_tensor(X_test)
    y_train, y_val, y_test = to_tensor(y_train), to_tensor(y_val), to_tensor(y_test)
    # create DataLoader
    train_dataset = TensorDataset(X_train, y_train)
    val_dataset = TensorDataset(X_val, y_val)
    test_dataset = TensorDataset(X_test, y_test)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

    lstm_model = LSTMModel(LSTM_input_size, LSTM_hidden_size, LSTM_num_layers, output_seq_len)
    lstm_model.to(device)

    epochs=200
    learning_rate = 0.001
    criterion = nn.MSELoss()
    optimizer = optim.Adam(lstm_model.parameters(), lr=learning_rate)
    epoch_logs=[]
    #input_feature will not input into the model
    input_feature = 0#0:WT,1:pH,2:DO,3:EC,4:TSS,5:CODMN,6:AN,7:TP,8:TN

    LSTM_train_and_evaluate(lstm_model, train_loader, val_loader, criterion, optimizer, epochs, target_site_index, input_feature)
    predictions, actuals = LSTM_evaluate(lstm_model, test_loader, target_site_index, input_feature)
    #Calculate accuracy
    # 计算精度
    rmse, r2, mae = calculate_metrics(predictions, actuals)
    #result
    # 输出结果
    print(f"predictions shape: {predictions.shape}")
    print(f"actuals shape: {actuals.shape}")
    print(f"Test RMSE: {rmse:.4f}")
    print(f"Test R2: {r2:.4f}")
    print(f"Test MAE: {mae:.4f}")
    feature_importances = np.mean(pd.DataFrame(LSTM_feature_importance_analysis(lstm_model, test_loader, target_site_index, input_feature, baseline='zero')),axis= 0)
    print(feature_importances)

    all_output_metrics.append([rmse, r2, mae])
    all_output_importance.append(pd.DataFrame({f"N{model_sites}" : feature_importances.values}, index = ['WT','pH','DO','EC','TSS','CODMn','AN','TP','TN']))
    epoch_logs = pd.DataFrame(epoch_logs,columns=['Epoch','Train_Loss','Val_loss','Time'])
    epoch_logs.to_csv(f"LSTM_logs_1siteallfeature_batch_site{model_sites}.csv",index=False)

result_output_metrics = pd.DataFrame(all_output_metrics,columns=['RMSE','R2','MAE'])
result_output_importance = pd.concat(all_output_importance,axis = 1)

result_output_metrics.to_csv(f"LSTM_1siteallfeature_batch_metrics_.csv",index=False)
result_output_importance.to_csv(f"LSTM_1siteallfeature_logs_one_batch_importance.csv",index=False)
#['WT','pH','DO','EC','TSS','CODMn','AN','TP','TN']

Epoch 1/200, Train Loss: 2.6202, Val Loss: 1.1323
Epoch 2/200, Train Loss: 0.7865, Val Loss: 0.8607
Epoch 3/200, Train Loss: 0.6741, Val Loss: 0.9804
Epoch 4/200, Train Loss: 0.6029, Val Loss: 0.9747
Epoch 5/200, Train Loss: 0.5386, Val Loss: 0.8182
Epoch 6/200, Train Loss: 0.4953, Val Loss: 0.6205
Epoch 7/200, Train Loss: 0.4408, Val Loss: 0.8804
Epoch 8/200, Train Loss: 0.4180, Val Loss: 0.9238
Epoch 9/200, Train Loss: 0.3838, Val Loss: 1.0262
Epoch 10/200, Train Loss: 0.3611, Val Loss: 0.6079
Epoch 11/200, Train Loss: 0.3491, Val Loss: 0.7814
Epoch 12/200, Train Loss: 0.3197, Val Loss: 0.9267
Epoch 13/200, Train Loss: 0.3162, Val Loss: 0.8959
Epoch 14/200, Train Loss: 0.2977, Val Loss: 0.6195
Epoch 15/200, Train Loss: 0.2878, Val Loss: 0.9947
Epoch 16/200, Train Loss: 0.2690, Val Loss: 1.0277
Epoch 17/200, Train Loss: 0.2658, Val Loss: 0.8459
Epoch 18/200, Train Loss: 0.2550, Val Loss: 0.5891
Epoch 19/200, Train Loss: 0.2444, Val Loss: 0.6369
Epoch 20/200, Train Loss: 0.2386, Val Lo

Epoch 12/200, Train Loss: 0.7660, Val Loss: 1.5407
Epoch 13/200, Train Loss: 0.7577, Val Loss: 1.5868
Epoch 14/200, Train Loss: 0.7373, Val Loss: 1.5623
Epoch 15/200, Train Loss: 0.7341, Val Loss: 1.6910
Epoch 16/200, Train Loss: 0.7194, Val Loss: 1.9042
Epoch 17/200, Train Loss: 0.7218, Val Loss: 2.0527
Epoch 18/200, Train Loss: 0.7068, Val Loss: 1.7841
Epoch 19/200, Train Loss: 0.6888, Val Loss: 1.5900
Epoch 20/200, Train Loss: 0.6876, Val Loss: 1.8324
Epoch 21/200, Train Loss: 0.6781, Val Loss: 1.9716
Early stopping triggered.
Test MSE: 1.5480
predictions shape: (6114, 1)
actuals shape: (6114, 1)
Test RMSE: 1.2442
Test R2: 0.3697
Test MAE: 1.0676
0    0.882222
1    0.267818
2    1.741739
3    0.783647
4    0.148683
5    0.004911
6    0.001696
7    0.382953
dtype: float32
Epoch 1/200, Train Loss: 2.0229, Val Loss: 1.8043
Epoch 2/200, Train Loss: 1.5061, Val Loss: 1.8709
Epoch 3/200, Train Loss: 1.4681, Val Loss: 1.6530
Epoch 4/200, Train Loss: 1.4390, Val Loss: 1.6047
Epoch 5/200, Tr

In [33]:
#test
# 模型超参数
LSTM_input_size = 8
LSTM_hidden_size = 64  # LSTM 隐藏层大小
LSTM_num_layers = 2

input_seq_len = 72  # 输入序列长度
output_seq_len = 1  # 输出序列长度
target_site_index = 0  # 目标站点的索引（例如 站点0）
target_col = 2  # 目标特征列的索引（例如 特征0）
batch_size = 16

X, y = create_sequences(data, input_seq_len, output_seq_len, target_site_index, target_col)
y = y.squeeze((-1,-2))
# 划分训练集和测试集
train_size, val_size, test_size = 0.7, 0.1, 0.2
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=test_size/(train_size+val_size+test_size), shuffle=False)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=val_size/(train_size+val_size), shuffle=False)
X_train, X_val, X_test = to_tensor(X_train), to_tensor(X_val), to_tensor(X_test)
y_train, y_val, y_test = to_tensor(y_train), to_tensor(y_val), to_tensor(y_test)
# 创建DataLoader
train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)
test_dataset = TensorDataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

lstm_model = LSTMModel(LSTM_input_size, LSTM_hidden_size, LSTM_num_layers, output_seq_len)
lstm_model.to(device)

epochs=200
learning_rate = 0.001
criterion = nn.MSELoss()
optimizer = optim.Adam(lstm_model.parameters(), lr=learning_rate)
epoch_logs=[]
input_feature = [0,1,3,4,5,6,7,8]#0水温	1pH	2溶解氧	3电导率	4浊度	5高锰酸盐	6氨氮	7总磷	8总氮

LSTM_train_and_evaluate(lstm_model, train_loader, val_loader, criterion, optimizer, epochs, target_site_index, input_feature)
predictions, actuals = LSTM_evaluate(lstm_model, test_loader, target_site_index, input_feature)
# 计算精度
rmse, r2, mae = calculate_metrics(predictions, actuals)
# 输出结果
print(f"predictions shape: {predictions.shape}")
print(f"actuals shape: {actuals.shape}")
print(f"Test RMSE: {rmse:.4f}")
print(f"Test R2: {r2:.4f}")
print(f"Test MAE: {mae:.4f}")

#['WT','pH','DO','EC','TSS','CODMn','AN','TP','TN']

Epoch 1/200, Train Loss: 2.6603, Val Loss: 1.2154
Epoch 2/200, Train Loss: 0.8296, Val Loss: 1.4550
Epoch 3/200, Train Loss: 0.6991, Val Loss: 1.2598
Epoch 4/200, Train Loss: 0.6079, Val Loss: 0.8132
Epoch 5/200, Train Loss: 0.5373, Val Loss: 0.8525
Epoch 6/200, Train Loss: 0.4523, Val Loss: 1.0703
Epoch 7/200, Train Loss: 0.4107, Val Loss: 0.7283
Epoch 8/200, Train Loss: 0.3748, Val Loss: 1.0407
Epoch 9/200, Train Loss: 0.3367, Val Loss: 0.8848
Epoch 10/200, Train Loss: 0.3039, Val Loss: 0.7364
Epoch 11/200, Train Loss: 0.2805, Val Loss: 0.9000
Epoch 12/200, Train Loss: 0.2529, Val Loss: 0.6937
Epoch 13/200, Train Loss: 0.2374, Val Loss: 0.6792
Epoch 14/200, Train Loss: 0.2414, Val Loss: 0.7130
Epoch 15/200, Train Loss: 0.2234, Val Loss: 0.6951
Epoch 16/200, Train Loss: 0.2094, Val Loss: 0.6312
Epoch 17/200, Train Loss: 0.2037, Val Loss: 0.6833
Epoch 18/200, Train Loss: 0.2113, Val Loss: 0.5691
Epoch 19/200, Train Loss: 0.1858, Val Loss: 0.8292
Epoch 20/200, Train Loss: 0.2012, Val Lo

In [34]:
feature_importances = np.mean(pd.DataFrame(LSTM_feature_importance_analysis(lstm_model, test_loader, target_site_index, input_feature, baseline='zero')),axis= 0)
print(feature_importances)
# max(data[:,4,3])

0    2.391290
1    2.771468
2    1.032951
3    0.758217
4    0.344273
5    0.086456
6    0.011174
7    0.401491
dtype: float32


## 所有站点单特征all site single feature

In [19]:
#所有站点单特征循环all site single feature
#change the Define function input:X_origin[:,:,:,input_feature].squeeze(),LSTM_input_size = 9
all_output_metrics = []
all_output_importance = []

for model_sites in range(9):
    output_metrics = []
    output_importance = []
    for model_input_feature in range(9):
        #Model hyperparameters
        # 模型超参数
        LSTM_input_size = 9
        LSTM_hidden_size = 64  
        LSTM_num_layers = 2

        input_seq_len = 72  # 输入序列长度,Input sequence length
        output_seq_len = 1  # 输出序列长度,output sequence length
        #target_site_index will note input into the model
        target_site_index = model_sites  # 目标站点（例如 站点0）,Target site (e.g. site 0)
        target_col = 2  # 目标特征（例如 特征0）,Target feature (e.g. feature 0)
        batch_size = 16

        X, y = create_sequences(data, input_seq_len, output_seq_len, target_site_index, target_col)
        y = y.squeeze((-1,-2))
        #Divide the training set，validation set and test set 7:1:2
        # 划分训练集和测试集7:1:2
        train_size, val_size, test_size = 0.7, 0.1, 0.2
        X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=test_size/(train_size+val_size+test_size), shuffle=False)
        X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=val_size/(train_size+val_size), shuffle=False)
        X_train, X_val, X_test = to_tensor(X_train), to_tensor(X_val), to_tensor(X_test)
        y_train, y_val, y_test = to_tensor(y_train), to_tensor(y_val), to_tensor(y_test)
        # create DataLoader
        train_dataset = TensorDataset(X_train, y_train)
        val_dataset = TensorDataset(X_val, y_val)
        test_dataset = TensorDataset(X_test, y_test)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

        lstm_model = LSTMModel(LSTM_input_size, LSTM_hidden_size, LSTM_num_layers, output_seq_len)
        lstm_model.to(device)

        epochs=200
        learning_rate = 0.001
        criterion = nn.MSELoss()
        optimizer = optim.Adam(lstm_model.parameters(), lr=learning_rate)
        epoch_logs=[]
        input_feature = model_input_feature#0:WT,1:pH,2:DO,3:EC,4:TSS,5:CODMN,6:AN,7:TP,8:TN

        LSTM_train_and_evaluate(lstm_model, train_loader, val_loader, criterion, optimizer, epochs, target_site_index, input_feature)
        predictions, actuals = LSTM_evaluate(lstm_model, test_loader, target_site_index, input_feature)
        #Calculate accuracy
        # 计算精度
        rmse, r2, mae = calculate_metrics(predictions, actuals)
        #result
        # 输出结果
        print(f"predictions shape: {predictions.shape}")
        print(f"actuals shape: {actuals.shape}")
        print(f"Test RMSE: {rmse:.4f}")
        print(f"Test R2: {r2:.4f}")
        print(f"Test MAE: {mae:.4f}")
        feature_importances = np.mean(pd.DataFrame(LSTM_feature_importance_analysis(lstm_model, test_loader, target_site_index, input_feature, baseline='zero')),axis= 0)
        print(feature_importances)
        
        feature_importanes= pd.DataFrame({f"F{model_input_feature}" : feature_importances.values}, index = [f'N{s}' for s in range(9)])
        
        output_metrics.append([rmse, r2, mae])
        output_importance.append(feature_importances)
        epoch_logs = pd.DataFrame(epoch_logs,columns=['Epoch','Train_Loss','Val_loss','Time'])
        epoch_logs.to_csv(f"LSTM_logs_allsite_1feature_batch_site{model_sites}_feature{model_input_feature}.csv",index=False)
         
    all_output_metrics.append(pd.DataFrame(output_metrics,columns=['RMSE','R2','MAE'], index = ['WT','pH','DO','EC','TSS','CODMn','AN','TP','TN']))
    all_output_importance = pd.concat(output_importance,axis = 1)
    all_output_importance.to_csv(f"LSTM_al1site_1feature_batch_importance_site{model_sites}.csv",index=False)

result_output_metrics = pd.concat(all_output_metrics,axis = 1)

result_output_metrics.to_csv(f"LSTM_al1site_1feature_batch_metrics.csv",index=False)

# all_output_metrics.to_csv(f"LSTM_allsite_1feature_batch_metrics备用查看数据.csv",index=False)
#['WT','pH','DO','EC','TSS','CODMn','AN','TP','TN']

Epoch 1/200, Train Loss: 3.0030, Val Loss: 0.9157
Epoch 2/200, Train Loss: 0.9191, Val Loss: 0.9055
Epoch 3/200, Train Loss: 0.8260, Val Loss: 1.0826
Epoch 4/200, Train Loss: 0.7328, Val Loss: 1.2477
Epoch 5/200, Train Loss: 0.6440, Val Loss: 1.2438
Epoch 6/200, Train Loss: 0.5811, Val Loss: 1.4037
Epoch 7/200, Train Loss: 0.5245, Val Loss: 1.6788
Epoch 8/200, Train Loss: 0.4472, Val Loss: 1.8964
Epoch 9/200, Train Loss: 0.4101, Val Loss: 1.8230
Epoch 10/200, Train Loss: 0.3665, Val Loss: 1.6289
Epoch 11/200, Train Loss: 0.4091, Val Loss: 1.5309
Epoch 12/200, Train Loss: 0.3304, Val Loss: 1.5466
Early stopping triggered.
Test MSE: 1.1234
predictions shape: (6114, 1)
actuals shape: (6114, 1)
Test RMSE: 1.0599
Test R2: 0.5329
Test MAE: 0.7735
0    2.296097
1    1.228502
2    0.843662
3    1.298746
4    1.704691
5    0.666102
6    0.990321
7    0.828745
dtype: float32
Epoch 1/200, Train Loss: 2.3002, Val Loss: 0.9018
Epoch 2/200, Train Loss: 0.7171, Val Loss: 0.6982
Epoch 3/200, Train Los

In [16]:
#test
#所有站点单特征
#所有站点单特征输入为X_origin[:,:,:,input_feature].squeeze(),LSTM_input_size = 9


model_sites = 2
LSTM_input_size = 8
LSTM_hidden_size = 64  # LSTM 隐藏层大小
LSTM_num_layers = 2

input_seq_len = 72  # 输入序列长度
output_seq_len = 1  # 输出序列长度
target_site_index = model_sites  # 目标站点的索引（例如 站点0）
target_col = 2  # 目标特征列的索引（例如 特征0）
batch_size = 16

X, y = create_sequences(data, input_seq_len, output_seq_len, target_site_index, target_col)
y = y.squeeze((-1,-2))
# 划分训练集和测试集
train_size, val_size, test_size = 0.7, 0.1, 0.2
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=test_size/(train_size+val_size+test_size), shuffle=False)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=val_size/(train_size+val_size), shuffle=False)
X_train, X_val, X_test = to_tensor(X_train), to_tensor(X_val), to_tensor(X_test)
y_train, y_val, y_test = to_tensor(y_train), to_tensor(y_val), to_tensor(y_test)
# 创建DataLoader
train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)
test_dataset = TensorDataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

lstm_model = LSTMModel(LSTM_input_size, LSTM_hidden_size, LSTM_num_layers, output_seq_len)
lstm_model.to(device)

epochs=200
learning_rate = 0.001
criterion = nn.MSELoss()
optimizer = optim.Adam(lstm_model.parameters(), lr=learning_rate)
epoch_logs=[]
input_feature = 2#0水温	1pH	2溶解氧	3电导率	4浊度	5高锰酸盐	6氨氮	7总磷	8总氮

LSTM_train_and_evaluate(lstm_model, train_loader, val_loader, criterion, optimizer, epochs, target_site_index, input_feature)
predictions, actuals = LSTM_evaluate(lstm_model, test_loader, target_site_index, input_feature)
# 计算精度
rmse, r2, mae = calculate_metrics(predictions, actuals)
# 输出结果
print(f"predictions shape: {predictions.shape}")
print(f"actuals shape: {actuals.shape}")
print(f"Test RMSE: {rmse:.4f}")
print(f"Test R2: {r2:.4f}")
print(f"Test MAE: {mae:.4f}")
feature_importances = np.mean(pd.DataFrame(LSTM_feature_importance_analysis(lstm_model, test_loader, target_site_index, input_feature, baseline='zero')),axis= 0)
print(feature_importances)


In [18]:
input_site = [4,8]
lstm_model = LSTMModel(LSTM_input_size, LSTM_hidden_size, LSTM_num_layers, output_seq_len)
lstm_model.to(device)

epochs=200
learning_rate = 0.001
criterion = nn.MSELoss()
optimizer = optim.Adam(lstm_model.parameters(), lr=learning_rate)
epoch_logs=[]
input_feature = 2#0水温	1pH	2溶解氧	3电导率	4浊度	5高锰酸盐	6氨氮	7总磷	8总氮

LSTM_train_and_evaluate(lstm_model, train_loader, val_loader, criterion, optimizer, epochs, input_site, input_feature)
predictions, actuals = LSTM_evaluate(lstm_model, test_loader, input_site, input_feature)
# 计算精度
rmse, r2, mae = calculate_metrics(predictions, actuals)
# 输出结果
print(f"predictions shape: {predictions.shape}")
print(f"actuals shape: {actuals.shape}")
print(f"Test RMSE: {rmse:.4f}")
print(f"Test R2: {r2:.4f}")
print(f"Test MAE: {mae:.4f}")
feature_importances = np.mean(pd.DataFrame(LSTM_feature_importance_analysis(lstm_model, test_loader, target_site_index, input_feature, baseline='zero')),axis= 0)
print(feature_importances)

Epoch 1/200, Train Loss: 2.3339, Val Loss: 1.3148
Epoch 2/200, Train Loss: 1.2836, Val Loss: 1.4694
Epoch 3/200, Train Loss: 1.2586, Val Loss: 1.4299
Epoch 4/200, Train Loss: 1.2340, Val Loss: 1.5982
Epoch 5/200, Train Loss: 1.2124, Val Loss: 1.6321
Epoch 6/200, Train Loss: 1.1877, Val Loss: 1.5310
Epoch 7/200, Train Loss: 1.1591, Val Loss: 2.0657
Epoch 8/200, Train Loss: 1.1723, Val Loss: 1.9402
Epoch 9/200, Train Loss: 1.0989, Val Loss: 1.8666
Epoch 10/200, Train Loss: 1.0318, Val Loss: 2.1865
Epoch 11/200, Train Loss: 0.9942, Val Loss: 1.6020
Early stopping triggered.
Test MSE: 1.9190
predictions shape: (6114, 1)
actuals shape: (6114, 1)
Test RMSE: 1.3853
Test R2: 0.6543
Test MAE: 1.1102


RuntimeError: input.size(-1) must be equal to input_size. Expected 2, got 72

## 所有站点所有特征all site all feature

In [49]:
#所有站点所有特征循环all site all feature 
#change the Define function input:X_origin[:,:,:,:].squeeze(),X_origin = X_origin.reshape(16,72,81),LSTM_input_size = 81
all_output_metrics = []
all_output_importance = []

for model_sites in range(9):
    # 模型超参数
    LSTM_input_size = 81
    LSTM_hidden_size = 64  # LSTM 隐藏层大小
    LSTM_num_layers = 2

    input_seq_len = 72  # 输入序列长度,Input sequence length
    output_seq_len = 1  # 输出序列长度,output sequence length
    #target_site_index will note input into the model
    target_site_index = model_sites  # 目标站点（例如 站点0）,Target site (e.g. site 0)
    target_col = 2  # 目标特征（例如 特征0）,Target feature (e.g. feature 0)
    batch_size = 16

    X, y = create_sequences(data, input_seq_len, output_seq_len, target_site_index, target_col)
    y = y.squeeze((-1,-2))
    #Divide the training set，validation set and test set 7:1:2
    # 划分训练集和测试集
    train_size, val_size, test_size = 0.7, 0.1, 0.2
    X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=test_size/(train_size+val_size+test_size), shuffle=False)
    X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=val_size/(train_size+val_size), shuffle=False)
    X_train, X_val, X_test = to_tensor(X_train), to_tensor(X_val), to_tensor(X_test)
    y_train, y_val, y_test = to_tensor(y_train), to_tensor(y_val), to_tensor(y_test)
    # create DataLoader
    train_dataset = TensorDataset(X_train, y_train)
    val_dataset = TensorDataset(X_val, y_val)
    test_dataset = TensorDataset(X_test, y_test)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

    lstm_model = LSTMModel(LSTM_input_size, LSTM_hidden_size, LSTM_num_layers, output_seq_len)
    lstm_model.to(device)

    epochs=200
    learning_rate = 0.001
    criterion = nn.MSELoss()
    optimizer = optim.Adam(lstm_model.parameters(), lr=learning_rate)
    epoch_logs=[]
    #input_feature will not input into the model
    input_feature = 0#0:WT,1:pH,2:DO,3:EC,4:TSS,5:CODMN,6:AN,7:TP,8:TN

    LSTM_train_and_evaluate(lstm_model, train_loader, val_loader, criterion, optimizer, epochs, target_site_index, input_feature)
    predictions, actuals = LSTM_evaluate(lstm_model, test_loader, target_site_index, input_feature)
    #Calculate accuracy
    # 计算精度
    rmse, r2, mae = calculate_metrics(predictions, actuals)
    #result
    # 输出结果
    print(f"predictions shape: {predictions.shape}")
    print(f"actuals shape: {actuals.shape}")
    print(f"Test RMSE: {rmse:.4f}")
    print(f"Test R2: {r2:.4f}")
    print(f"Test MAE: {mae:.4f}")
    feature_importances = np.mean(pd.DataFrame(LSTM_feature_importance_analysis(lstm_model, test_loader, target_site_index, input_feature, baseline='zero')),axis= 0)
    print(feature_importances)

    all_output_metrics.append([rmse, r2, mae])
    all_output_importance.append(pd.DataFrame({f"N{model_sites}" : feature_importances.values}, index=[f'F{s}' for s in range(81)]))
    epoch_logs = pd.DataFrame(epoch_logs,columns=['Epoch','Train_Loss','Val_loss','Time'])
    epoch_logs.to_csv(f"LSTM_logs_allsiteallfeature_batch_site{model_sites}.csv",index=False)

result_output_metrics = pd.DataFrame(output_metrics,columns=['RMSE','R2','MAE'])
result_output_importance = pd.concat(all_output_importance,axis = 1)

result_output_metrics.to_csv(f"LSTM_allsiteallfeature_batch_metrics_.csv",index=False)
result_output_importance.to_csv(f"LSTM_1siteallfeature_logs_one_batch_importance.csv",index=False)
# all_output_importance.to_csv(f"LSTM_allsiteallfeature_logs_one_batch_importance备用查看数据.csv",index=False)
#['WT','pH','DO','EC','TSS','CODMn','AN','TP','TN']

Epoch 1/200, Train Loss: 3.2107, Val Loss: 2.0822
Epoch 2/200, Train Loss: 1.3884, Val Loss: 2.3025
Epoch 3/200, Train Loss: 1.1137, Val Loss: 1.7821
Epoch 4/200, Train Loss: 0.9477, Val Loss: 1.9034
Epoch 5/200, Train Loss: 0.8750, Val Loss: 2.1305
Epoch 6/200, Train Loss: 0.8443, Val Loss: 2.1866
Epoch 7/200, Train Loss: 0.7661, Val Loss: 1.9526
Epoch 8/200, Train Loss: 0.7565, Val Loss: 2.3212
Epoch 9/200, Train Loss: 0.7115, Val Loss: 1.7734
Epoch 10/200, Train Loss: 0.6662, Val Loss: 1.5127
Epoch 11/200, Train Loss: 0.6588, Val Loss: 1.9886
Epoch 12/200, Train Loss: 0.6176, Val Loss: 1.8938
Epoch 13/200, Train Loss: 0.6002, Val Loss: 1.6692
Epoch 14/200, Train Loss: 0.6049, Val Loss: 1.7951
Epoch 15/200, Train Loss: 0.5733, Val Loss: 1.6322
Epoch 16/200, Train Loss: 0.5538, Val Loss: 1.4850
Epoch 17/200, Train Loss: 0.5407, Val Loss: 1.6763
Epoch 18/200, Train Loss: 0.5204, Val Loss: 1.6655
Epoch 19/200, Train Loss: 0.5245, Val Loss: 1.4998
Epoch 20/200, Train Loss: 0.5000, Val Lo

0     0.088548
1     0.034884
2     0.059406
3     0.276840
4     0.059199
        ...   
76    0.075109
77    0.018241
78    0.008419
79    0.000306
80    0.022159
Length: 81, dtype: float32
Epoch 1/200, Train Loss: 1.8353, Val Loss: 1.5274
Epoch 2/200, Train Loss: 1.2031, Val Loss: 1.3142
Epoch 3/200, Train Loss: 1.1226, Val Loss: 1.2441
Epoch 4/200, Train Loss: 1.0578, Val Loss: 0.9333
Epoch 5/200, Train Loss: 1.0151, Val Loss: 1.5136
Epoch 6/200, Train Loss: 0.9584, Val Loss: 1.3747
Epoch 7/200, Train Loss: 0.9032, Val Loss: 1.3012
Epoch 8/200, Train Loss: 0.8798, Val Loss: 1.2798
Epoch 9/200, Train Loss: 0.8540, Val Loss: 1.5454
Epoch 10/200, Train Loss: 0.8193, Val Loss: 1.3789
Epoch 11/200, Train Loss: 0.7910, Val Loss: 1.0549
Epoch 12/200, Train Loss: 0.7684, Val Loss: 1.3401
Epoch 13/200, Train Loss: 0.7475, Val Loss: 1.4703
Epoch 14/200, Train Loss: 0.7210, Val Loss: 1.1385
Early stopping triggered.
Test MSE: 1.2508
predictions shape: (6114, 1)
actuals shape: (6114, 1)
Test R

## 其他other

In [ ]:
data_iter = iter(test_loader)  # 创建一个迭代器
X_sample, y_sample = next(data_iter)  # 获取一个批次数据
print(X_sample.shape)